In [1]:
import board
import neopixel
import time
import numpy as np

num_pixels = 100
ORDER = neopixel.RGB

pixels = neopixel.NeoPixel(board.D21, num_pixels, auto_write=False, pixel_order=ORDER)

In [2]:
# E.g. Set all pixels off 
pixels.fill((0, 0, 0))

In [3]:
loop_starters = [0, 10, 30, 50, 70]

In [4]:
# Test loop_starters
for i in range(0, len(loop_starters)):
    pixels[loop_starters[i]] = (255, 255, 255)
pixels.show()
time.sleep(2)
pixels.fill((0, 0, 0))
pixels.show()

In [5]:
loop_starters = loop_starters + [num_pixels,]

In [6]:
def colour_check(colour):
    for i in range(0, 3):
        if colour[i] < 0:
            colour[i] = 0
        elif colour[i] > 255:
            colour[i] = 255
        else:
            continue
    return colour

        
def sine_wave(pixels, loop_starters, amplitute, phase_shift, dc_offset, colour):
    # Input a value 0 to 255 to get a color value.
    for i in range(0, len(loop_starters)-1):
        actuation = amplitute*np.sin(2*np.pi*(i+1)/(len(loop_starters))+phase_shift)+dc_offset
        relative_pos = (actuation+1)/(2)
        pos = round((loop_starters[i+1]-loop_starters[i])*relative_pos+loop_starters[i])
        if pos < 0:
            pos = 0
        if pos >= num_pixels:
            pos = num_pixels - 1
        new_colour = colour_check([sum(tpl) for tpl in zip (pixels[pos], colour)])
        pixels[pos] = new_colour
    return pixels

In [7]:
steps = 10

In [8]:
sine_waves = [{"amplitute_range": [-1, 1], 
               "phase_shift_range": [0, 0],
               "dc_offset_range": [0, 0],
               "colour": (255, 0, 0)},
              {"amplitute_range": [0.7, 0.7], 
               "phase_shift_range": [0, 0],
               "dc_offset_range": [-0.2, 0.2],
               "colour": (0, 255, 0)},
              {"amplitute_range": [-0.6, 0.6], 
               "phase_shift_range": [0, 0],
               "dc_offset_range": [0, 0],
               "colour": (0, 0, 255)},
              {"amplitute_range": [-0.9, -0.9], 
               "phase_shift_range": [0, 0],
               "dc_offset_range": [-0.07, 0.07],
               "colour": (128, 128, 0)}]

In [9]:
for i in range(0, len(sine_waves)):
    sine_waves[i]["amplitutes"] = np.concatenate((np.linspace(sine_waves[i]["amplitute_range"][0],
                                                              sine_waves[i]["amplitute_range"][1],
                                                              num=steps)[:-1],
                                                  np.linspace(sine_waves[i]["amplitute_range"][1],
                                                              sine_waves[i]["amplitute_range"][0],
                                                              num=steps)[:-1]),
                                                 axis=0)
    sine_waves[i]["phase_shifts"] = np.concatenate((np.linspace(sine_waves[i]["phase_shift_range"][0],
                                                                sine_waves[i]["phase_shift_range"][1],
                                                                num=steps)[:-1],
                                                    np.linspace(sine_waves[i]["phase_shift_range"][1],
                                                                sine_waves[i]["phase_shift_range"][0],
                                                                num=steps)[:-1]),
                                                 axis=0)                                            
    sine_waves[i]["dc_offsets"] = np.concatenate((np.linspace(sine_waves[i]["dc_offset_range"][0],
                                                              sine_waves[i]["dc_offset_range"][1],
                                                              num=steps)[:-1],
                                                  np.linspace(sine_waves[i]["dc_offset_range"][1],
                                                              sine_waves[i]["dc_offset_range"][0],
                                                              num=steps)[:-1]),
                                                  axis=0)

In [10]:
start_step = 0
while True:
    pixels.fill((0, 0, 0))
    for i in range(0, len(sine_waves)):
        pixels = sine_wave(pixels,
                           loop_starters,
                           sine_waves[i]["amplitutes"][start_step],
                           sine_waves[i]["phase_shifts"][start_step],
                           sine_waves[i]["dc_offsets"][start_step],
                           sine_waves[i]["colour"])
    pixels.show()
    start_step += 1
    if start_step >= 2*(steps-1):
        start_step = 0
    time.sleep(0.3)

KeyboardInterrupt: 

In [ ]:
# Turn off pixels with pixels.deinit()

In [11]:
pixels.deinit()